In [152]:
import polars as pl
import pandas as pd
import shutil
import os
import fnmatch
from datetime import datetime
from typing import List, Optional
import pyarrow as pa
from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
import numpy as np



In [153]:
# Column name constants
WAREHOUSE = "Almacen"
DATE = "Fecha"
CUSTOMER = "Cliente"
SELLER = "Vendedor"
PRODUCT_CONCAT = "ProdConcat"
QUANTITY = "Cantidad"
SALE_PRICE = "PrecioVenta"
PAYMENT_METHODS = "Metodos De Pago"
NO_MOV = "NoMov"

DIRECTORY = "../data/sales"
KEYWORDS = "Ventas por Tickets"
SALES_COLUMNS =  [
    WAREHOUSE,
    NO_MOV,
    DATE,
    CUSTOMER,
    SELLER,
    PRODUCT_CONCAT,
    QUANTITY,
    SALE_PRICE,
    PAYMENT_METHODS
]
OUTPUT_PATH = "../output"
OUTPUT_FILE_NAME = "salesReportMerged"
# EDA Metric constants
TOTAL_SALES = "TotalSales"
SALES_MEAN = "PROMEDIO"
SALES_MEDIAN = "MEDIANA"
SALES_STDDEV = "DESVIACION ESTANDAR"
SALES_MIN = "VENTA MINIMA"
SALES_MAX = "VENTA MAXIMA"


In [154]:
def readExcelToDataFrame(path: str, sheetName: Optional[str] = None) -> pl.DataFrame:
    """
    Reads an Excel (.xlsx) file and returns it as a Polars DataFrame.

    :param path: Full path to the Excel file
    :param sheetName: Name of the sheet to read (default is the first sheet)
    :return: A pl.DataFrame object

    Example usage:
    >>> df = readExcelToDataFrame("./data/salesReport.xlsx")
    >>> print(df)

    >>> dfSheet = readExcelToDataFrame("./data/salesReport.xlsx", sheetName="JanuarySales")
    >>> print(dfSheet)
    """
    if not path.endswith('.xlsx'):
        raise ValueError("The file must have a .xlsx extension.")
    
    if not os.path.exists(path):
        raise FileNotFoundError(f"The file '{path}' does not exist.")
    
    try:
        # Try using Polars native read_excel
        dfPolars = pl.read_excel(path, sheet_name=sheetName)
    except (AttributeError, TypeError):
        # Fallback if Polars version does not support read_excel or sheet_name argument
        dfPandas = pd.read_excel(path, sheet_name=sheetName)
        dfPolars = pl.from_pandas(dfPandas)

    return dfPolars

def unionDataFrames(dfList: List[pl.DataFrame]) -> pl.DataFrame:
    """
    Merges a list of Polars DataFrames into a single DataFrame.

    :param dfList: List of pl.DataFrame objects
    :return: A single pl.DataFrame resulting from concatenating all the DataFrames

    Example usage:
    >>> df1 = pl.DataFrame({"a": [1, 2], "b": [3, 4]})
    >>> df2 = pl.DataFrame({"a": [5, 6], "b": [7, 8]})
    >>> result = unionDataFrames([df1, df2])
    >>> print(result)
    """
    if not dfList:
        raise ValueError("The DataFrame list is empty.")
    
    if not all(isinstance(df, pl.DataFrame) for df in dfList):
        raise TypeError("All elements in the list must be of type pl.DataFrame.")
    
    return pl.concat(dfList)

def saveDataFrameToExcel(df: pl.DataFrame, path: str, fileName: str) -> None:
    """
    Converts a Polars DataFrame to Excel and saves it to the specified path.

    :param df: A pl.DataFrame object
    :param path: Directory path where the Excel file will be saved
    :param fileName: Output filename without extension

    Example usage:
    >>> df = pl.DataFrame({"a": [1, 2], "b": [3, 4]})
    >>> saveDataFrameToExcel(df, "./outputs", "myReport")
    """
    if not isinstance(df, pl.DataFrame):
        raise TypeError("The 'df' parameter must be a pl.DataFrame.")

    if not fileName.endswith('.xlsx'):
        fileName += '.xlsx'

    os.makedirs(path, exist_ok=True)

    # Step 1: Try converting normally
    try:
        dfPandas = df.to_pandas()
    except Exception as e:
        print(f"Warning: Direct to_pandas() failed. Trying manual conversion. Error: {e}")
        # Step 2: Manual fallback: recreate a Pandas DataFrame manually
        dfPandas = pd.DataFrame({col: df[col].to_list() for col in df.columns})

    # Step 3: Save as Excel
    outputPath = os.path.join(path, fileName)
    dfPandas.to_excel(outputPath, index=False)

def selectColumns(df: pl.DataFrame, columnsToKeep: List[str]) -> pl.DataFrame:
    """
    Selects specific columns from a Polars DataFrame and returns a new DataFrame.

    :param df: A pl.DataFrame object
    :param columnsToKeep: List of column names to keep
    :return: A pl.DataFrame with only the specified columns

    Example usage:
    >>> df = pl.DataFrame({
    ...     "name": ["Alice", "Bob", "Charlie"],
    ...     "age": [25, 30, 35],
    ...     "city": ["NYC", "LA", "Chicago"]
    ... })
    >>> selectedDf = selectColumns(df, ["name", "city"])
    >>> print(selectedDf)
    """
    if not isinstance(df, pl.DataFrame):
        raise TypeError("The 'df' parameter must be a pl.DataFrame.")
    
    if not all(isinstance(col, str) for col in columnsToKeep):
        raise TypeError("All elements in 'columnsToKeep' must be strings.")
    
    missingColumns = [col for col in columnsToKeep if col not in df.columns]
    if missingColumns:
        raise ValueError(f"The following columns are not present in the DataFrame: {missingColumns}")
    
    return df.select(columnsToKeep)

def replaceNaNWithZero(df: pl.DataFrame) -> pl.DataFrame:
    """
    Replaces NaN or null values with 0 in all numeric columns of a Polars DataFrame.

    :param df: A pl.DataFrame object to process
    :return: A new pl.DataFrame with NaN/null values replaced by 0 in numeric columns only

    Example usage:
    >>> df = pl.DataFrame({
    ...     "sales": [100, None, 300],
    ...     "profit": [50, 75, None],
    ...     "city": ["NYC", "LA", "Chicago"]
    ... })
    >>> updatedDf = replaceNaNWithZero(df)
    >>> print(updatedDf)
    """
    if not isinstance(df, pl.DataFrame):
        raise TypeError("The 'df' parameter must be a pl.DataFrame.")

    numericColumns = [col for col, dtype in zip(df.columns, df.dtypes) if dtype in (pl.Int8, pl.Int16, pl.Int32, pl.Int64, pl.UInt8, pl.UInt16, pl.UInt32, pl.UInt64, pl.Float32, pl.Float64)]
    
    df = df.with_columns(
        [pl.col(col).fill_null(0).alias(col) for col in numericColumns]
    )
    
    return df

def filterLastNDaysFromMaxDate(df: pl.DataFrame, days: int) -> pl.DataFrame:
    """
    Filters the DataFrame to keep only the last N days from the maximum registered date in the DATE column.

    :param df: A Polars DataFrame
    :param days: Number of days to go back from the latest date
    :return: A filtered Polars DataFrame
    """
    from datetime import timedelta

    if not isinstance(df, pl.DataFrame):
        raise TypeError("The 'df' parameter must be a Polars DataFrame.")

    if DATE not in df.columns:
        raise ValueError(f"The column '{DATE}' does not exist in the DataFrame.")

    # 1. Find the maximum date in the dataset
    maxDate = df.select(pl.col(DATE).max()).item()

    if not isinstance(maxDate, datetime):
        raise ValueError(f"The maximum value in column '{DATE}' is not a datetime.")

    # 2. Calculate cutoff date
    cutoffDate = maxDate - timedelta(days=days)

    # 3. Filter the DataFrame
    filteredDf = df.filter(pl.col(DATE) >= cutoffDate)

    return filteredDf





In [155]:
def findExcelFilesByMatch(directory: str, keyword: str, extension: str = "xlsx") -> List[str]:
    """
    Finds all files in a directory that contain a specific keyword and have the specified extension.

    :param directory: Directory path to search for files
    :param keyword: Keyword to match in filenames
    :param extension: File extension to match (default is 'xlsx')
    :return: List of matching filenames

    Example usage:
    >>> files = findExcelFilesByMatch("./data", "sales")
    >>> print(files)

    >>> csvFiles = findExcelFilesByMatch("./data", "sales", "csv")
    >>> print(csvFiles)
    """
    matchedFiles = []
    pattern = f"*{keyword}*.{extension}"
    
    for fileName in os.listdir(directory):
        if fnmatch.fnmatch(fileName, pattern):
            matchedFiles.append(fileName)

    return matchedFiles

def getCurrentTimestamp() -> str:
    """
    Returns the current date and time formatted as 'YYYY-MM-DDTHH-MM-SS'.

    :return: A string representing the current timestamp
    Example usage:
    >>> timestamp = getCurrentTimestamp()
    >>> print(timestamp)
    """
    return datetime.now().strftime("%Y-%m-%dT%H-%M-%S")

def createFolder(baseFolderName: str, basePath: str = ".", addTimestamp: bool = True) -> Optional[str]:
    """
    Creates a folder with the base name, optionally followed by the current timestamp.

    :param baseFolderName: Base name for the folder
    :param basePath: Directory where the folder will be created (default is the current directory)
    :param addTimestamp: Whether to append the current timestamp to the folder name (default is True)
    :return: Full path of the created folder or None if creation fails

    Example usage:
    >>> folderPath = createFolder("report")
    >>> print(folderPath)

    >>> folderPathNoTimestamp = createFolder("report", addTimestamp=False)
    >>> print(folderPathNoTimestamp)
    """
    try:
        fullFolderName = baseFolderName
        if addTimestamp:
            fullFolderName += f"_{getCurrentTimestamp()}"
        
        fullPath = os.path.join(basePath, fullFolderName)

        os.makedirs(fullPath, exist_ok=True)
        print(f"Folder created: {fullPath}")
        return fullPath
    except Exception as e:
        print(f"Error creating the folder: {e}")
        return None

def moveFilesToFolder(fileList: List[str], destinationFolder: str) -> None:
    """
    Moves a list of files to a destination folder.

    :param fileList: List of file paths to move
    :param destinationFolder: Path of the destination folder
    :return: None

    Example usage:
    >>> moveFilesToFolder(["./file1.txt", "./file2.txt"], "./backup")
    """
    try:
        # Create the destination folder if it doesn't exist
        if not os.path.exists(destinationFolder):
            createdFolder = createFolder(destinationFolder, basePath=".", addTimestamp=False)
            if createdFolder:
                destinationFolder = createdFolder
            else:
                raise Exception(f"Failed to create the destination folder: {destinationFolder}")

        for filePath in fileList:
            if os.path.isfile(filePath):
                destinationPath = os.path.join(destinationFolder, os.path.basename(filePath))
                shutil.move(filePath, destinationPath)
                print(f"File moved: {filePath} -> {destinationPath}")
            else:
                print(f"File does not exist: {filePath}")
    except Exception as e:
        print(f"Error moving files: {e}")

def validateFiles(fileList: List[str]) -> bool:
    """
    Validates that a list of file paths does not contain empty values.

    :param fileList: List of file paths
    :return: True if all files are valid, False if there are missing files

    Example usage:
    >>> isValid = validateFiles(["file1.xlsx", "file2.xlsx", ""])
    >>> print(isValid)
    """
    missingFiles = [filePath for filePath in fileList if filePath.strip() == ""]

    if missingFiles:
        print("Error: Missing required files for the report process.")
        return False

    print("All files are valid.")
    return True

In [156]:
def createMergedDataFrameFromExcelMatch(
    directory: str = "./",
    keyword: str = "",
    columns: Optional[List[str]] = None,
    sheetName: Optional[str] = None,
    saveToExcel: bool = False,
    outputPath: Optional[str] = None,
    outputFileName: str = "mergedReport"
) -> pl.DataFrame:
    """
    Searches for Excel files in a directory that match a keyword,
    reads them into DataFrames (optionally filtering columns and selecting a specific sheet),
    merges them into a single DataFrame, and optionally saves it as an Excel file.

    :param directory: Directory path to search for Excel files (default "./")
    :param keyword: Keyword to match in filenames
    :param columns: List of columns to select from each file (optional)
    :param sheetName: Name of the sheet to read (optional)
    :param saveToExcel: Whether to save the merged DataFrame to an Excel file (default False)
    :param outputPath: Directory where the Excel file will be saved (required if saveToExcel=True)
    :param outputFileName: Name of the output Excel file without extension (default "mergedReport")
    :return: A single merged pl.DataFrame from all successfully read files
    :raises FileNotFoundError: If no matching files are found
    :raises ValueError: If no DataFrames could be read or merged
     
    Example usage:
    >>> df = createMergedDataFrameFromExcelMatch("./data", "sales", ["id", "amount"], "January", True, "./outputs", "salesJanuary")
    >>> print(df)
    """
    # 1. Find matching Excel files
    matchingFiles = findExcelFilesByMatch(directory, keyword, extension="xlsx")
    
    if not matchingFiles:
        raise FileNotFoundError(f"No Excel files matching '{keyword}' were found in '{directory}'.")

    # 2. Read all files into DataFrames
    dataFrames = []
    for fileName in matchingFiles:
        fullPath = os.path.join(directory, fileName)
        try:
            df = readExcelToDataFrame(fullPath, sheetName=sheetName)
            if columns:
                df = selectColumns(df, columns)
            dataFrames.append(df)
        except Exception as e:
            print(f"Warning: Could not process file '{fileName}': {e}")

    # 3. Validate at least one DataFrame was read
    if not dataFrames:
        raise ValueError("No valid DataFrames could be created from the matching files.")

    # 4. Merge all DataFrames
    mergedDataFrame = unionDataFrames(dataFrames)

    # 5. Save to Excel if requested
    if saveToExcel:
        if outputPath is None:
            raise ValueError("outputPath must be provided when saveToExcel=True.")
        saveDataFrameToExcel(mergedDataFrame, outputPath, outputFileName)

    return mergedDataFrame


In [157]:
def cleanDateColumn(df: pl.DataFrame, dateColumnName: str) -> pl.DataFrame:
    """
    Converts the specified column in a DataFrame from string format to a proper Datetime type.

    :param df: A pl.DataFrame containing the date column
    :param dateColumnName: Name of the column to convert
    :return: A pl.DataFrame with the specified column converted to Datetime type
     
    Example usage:
    >>> dfCleaned = cleanDateColumn(dfSalesMerged, DATE)
    >>> print(dfCleaned)
    """
    if not isinstance(df, pl.DataFrame):
        raise TypeError("The 'df' parameter must be a pl.DataFrame.")

    if dateColumnName not in df.columns:
        raise ValueError(f"The column '{dateColumnName}' does not exist in the DataFrame.")

    return df.with_columns([
        pl.col(dateColumnName)
        .str.strptime(pl.Datetime, format="%b %e %Y %I:%M%p")
        .alias(dateColumnName)
    ])


In [ ]:
def saveFigureToPdf(fig, pdf: PdfPages) -> None:
    """
    Saves a matplotlib figure into a PDF file with a white background.

    :param fig: A matplotlib figure object
    :param pdf: A PdfPages object to save the figure
    """
    pdf.savefig(fig, facecolor='white', edgecolor='white')
    plt.close(fig)

def prepareSummaryStatistics(df: pl.DataFrame) -> pd.DataFrame:
    """
    Prepares summary statistics for the sales prices grouped by warehouse and ticket (Warehouse + NoMov):
    - Mean
    - Median
    - Standard Deviation
    - Minimum
    - Maximum
    All values are rounded to 2 decimal places.

    :param df: A Polars DataFrame
    :return: A Pandas DataFrame with the summary statistics
    """
    # 1. Aggregate total sale per warehouse and ticket (Warehouse + NoMov)
    ticketSales = df.group_by([WAREHOUSE, NO_MOV]).agg([
        pl.col(SALE_PRICE).sum().alias(TOTAL_SALES)
    ])

    # 2. Calculate summary statistics over the ticket totals
    summary = ticketSales.select([
        pl.col(TOTAL_SALES).mean().alias(SALES_MEAN),
        pl.col(TOTAL_SALES).median().alias(SALES_MEDIAN),
        pl.col(TOTAL_SALES).std().alias(SALES_STDDEV),
        pl.col(TOTAL_SALES).min().alias(SALES_MIN),
        pl.col(TOTAL_SALES).max().alias(SALES_MAX)
    ])
    
    # 3. Convert manually to Pandas DataFrame
    summaryDf = pd.DataFrame(summary.to_dict(as_series=False))
    
    # 4. Round all numeric values to 2 decimal places
    summaryDf = summaryDf.round(2)
    
    return summaryDf

def prepareTopSellers(
    df: pl.DataFrame,
    limit: int = 5,
    ascending: bool = False,
    days: int = 30
) -> pd.DataFrame:
    """
    Prepares the top or bottom sellers by total sales within the last 'days' from the latest recorded date.

    :param df: A Polars DataFrame containing sales data
    :param limit: Number of sellers to return (default 5)
    :param ascending: Sort in ascending order for bottom sellers, or descending for top sellers (default False)
    :param days: Number of days to look back from the latest date to filter sales data (default 30)
    :return: A Pandas DataFrame with the top or bottom sellers
    """
    # 1. Filter the DataFrame to keep only the last 'days' from the maximum date in the dataset
    df = filterLastNDaysFromMaxDate(df, days=days)

    # 2. Group by seller and calculate total sales
    topSellers = df.group_by(SELLER).agg([
        pl.col(SALE_PRICE).sum().alias(TOTAL_SALES)
    ]).sort(TOTAL_SALES, descending=not ascending)

    # 3. Take only the top or bottom 'limit' sellers
    topSellers = topSellers.head(limit)

    # 4. Manual conversion to Pandas DataFrame
    return pd.DataFrame(topSellers.to_dict(as_series=False))

def prepareSalesByDay(df: pl.DataFrame, days: int = 30) -> pd.DataFrame:
    """
    Aggregates total sales per day, limited to the most recent N days.

    :param df: A Polars DataFrame containing sales data
    :param days: Number of most recent days to include (default is 30)
    :return: A Pandas DataFrame with total sales aggregated by day
    """
    # 1. Filter the DataFrame to include only the last 'days' from the most recent date
    df = filterLastNDaysFromMaxDate(df, days=days)

    # 2. Group sales by date and calculate total sales per day
    salesByDay = df.group_by(pl.col(DATE).dt.date()).agg([
        pl.col(SALE_PRICE).sum().alias(TOTAL_SALES)
    ]).sort(DATE)

    # 3. Convert manually to Pandas DataFrame
    return pd.DataFrame(salesByDay.to_dict(as_series=False))

def prepareSalesDistributions(df: pl.DataFrame, days: int = 30) -> None:
    """
    Genera histogramas y diagramas de caja (boxplots) para los precios de venta,
    tanto en escala original como en escala logarítmica.
    Filtra los datos para incluir solo los últimos N días a partir de la fecha más reciente.
    Muestra etiquetas en valores reales, incluso para los gráficos logarítmicos.

    :param df: Un DataFrame de Polars
    :param days: Número de días recientes a considerar (por defecto 30)
    :yield: Figuras de Matplotlib listas para ser guardadas en PDF
    """
    import numpy as np

    # 1. Filtrar últimos N días
    df = filterLastNDaysFromMaxDate(df, days=days)

    # 2. Convertir a pandas
    salesData = pd.DataFrame(df.select(SALE_PRICE).to_dict(as_series=False))

    # 3. Histograma - Escala logarítmica con etiquetas reales
    salesDataLog = salesData.copy()
    salesDataLog[SALE_PRICE] = salesDataLog[SALE_PRICE].apply(lambda x: np.log1p(x))

    fig2, ax2 = plt.subplots(facecolor="white")
    salesDataLog.hist(bins=50, ax=ax2, color="salmon", edgecolor="black")
    ax2.set_title(f"Histograma (últimos {days} días)", fontsize=16)
    ax2.set_xlabel("Precio de Venta")
    ax2.set_ylabel("Frecuencia")

    # Etiquetas del eje X transformadas
    ticks = [0, 2, 4, 6, 8, 10]
    labels = [f"${int(np.expm1(t)):,}" for t in ticks]
    ax2.set_xticks(ticks)
    ax2.set_xticklabels(labels)

    plt.grid(True, linestyle="--", linewidth=0.5)
    plt.tight_layout()
    yield fig2

    # 4. Boxplot - Escala logarítmica con etiquetas reales
    fig4, ax4 = plt.subplots(facecolor="white")
    salesDataLog.boxplot(ax=ax4)
    ax4.set_title(f"Boxplot (últimos {days} días)", fontsize=16)
    ax4.set_ylabel("Precio de Venta")

    y_ticks = [0, 2, 4, 6, 8, 10]
    y_labels = [f"${int(np.expm1(y)):,}" for y in y_ticks]
    ax4.set_yticks(y_ticks)
    ax4.set_yticklabels(y_labels)

    plt.grid(True, linestyle="--", linewidth=0.5)
    plt.tight_layout()
    yield fig4

def generateSalesEDA(df: pl.DataFrame, outputPdfPath: str, days: int = 30) -> None:
    """
    Generates the Exploratory Data Analysis (EDA) for sales data and saves it to a PDF.

    :param df: A Polars DataFrame
    :param outputPdfPath: Path where the PDF will be saved
    """
    os.makedirs(os.path.dirname(outputPdfPath), exist_ok=True)
    pdf = PdfPages(outputPdfPath)
    
    # 1. Estadísticas Resumen
    summaryStats = prepareSummaryStatistics(df)
    summaryDict = summaryStats.to_dict(orient="records")[0]
    fig, ax = plt.subplots(figsize=(8.5, 11), facecolor="white")
    # Datos
    labels = list(summaryDict.keys())
    values = list(summaryDict.values())
    # Barras horizontales
    bars = ax.barh(labels, values, color="skyblue", edgecolor="black")
    # Escala log en X
    ax.set_xscale("log")
    # Etiquetas dentro de cada barra (valores reales con formato bonito)
    for bar, value in zip(bars, values):
        ax.text(value, bar.get_y() + bar.get_height() / 2,
                f"${int(value):,}", va="center", ha="left", fontsize=9, color="black")
    # Personalización
    plt.title("Estadísticas Resumen de Precios de Venta", fontsize=20)
    plt.xlabel("Valor", fontsize=14)
    # Etiquetas del eje X en formato real (aunque esté log)
    ticks = [1, 10, 100, 1_000, 10_000, 100_000]
    labels = [f"${tick:,}" for tick in ticks]
    ax.set_xticks(ticks)
    ax.set_xticklabels(labels)

    plt.grid(True, axis='x', linestyle="--", linewidth=0.5)
    plt.yticks(rotation=45, fontsize=6, ha="right", rotation_mode="anchor")
    plt.tight_layout()

    # Guardar al PDF
    saveFigureToPdf(fig, pdf)

    # 2. Ventas por día
    salesByDay = prepareSalesByDay(df, days=days)
    fig, ax = plt.subplots(facecolor="white")
    salesByDay.plot(x=DATE, y=TOTAL_SALES, kind="line", ax=ax)
    plt.title(f"Ventas Totales por Día (últimos {days} días)", fontsize=16)
    plt.xlabel("Fecha", fontsize=10)
    plt.ylabel("Ventas Totales", fontsize=12)
    plt.xticks(rotation=45)
    plt.grid()
    plt.tight_layout()
    saveFigureToPdf(fig, pdf)

    # 3. Mejores vendedores
    topBestSellers = prepareTopSellers(df, limit=20, ascending=False, days=days)
    fig, ax = plt.subplots(figsize=(8.5, 11), facecolor="white")
    ax.set_facecolor("white")
    topBestSellers.plot(x=SELLER, y=TOTAL_SALES, kind="bar", ax=ax, color="skyblue", edgecolor="black")
    plt.title(f"Top 20 Mejores Vendedores (últimos {days} días)", fontsize=20)
    plt.xlabel("Vendedor", fontsize=8)
    plt.ylabel("Ventas Totales", fontsize=14)
    plt.xticks(rotation=45, fontsize=6, ha="right", rotation_mode="anchor")
    plt.yticks(fontsize=12)
    plt.grid(True, color="grey", linestyle="--", linewidth=0.5)
    plt.tight_layout()
    saveFigureToPdf(fig, pdf)

    # 4. Peores vendedores
    topWorstSellers = prepareTopSellers(df, limit=20, ascending=True, days=days)
    fig, ax = plt.subplots(figsize=(8.5, 11), facecolor="white")
    ax.set_facecolor("white")
    topWorstSellers.plot(x=SELLER, y=TOTAL_SALES, kind="bar", ax=ax, color="skyblue", edgecolor="black")
    plt.title(f"Top 20 Peores Vendedores (últimos {days} días)", fontsize=20)
    plt.xlabel("Vendedor", fontsize=8)
    plt.ylabel("Ventas Totales", fontsize=14)
    plt.xticks(rotation=45, fontsize=6, ha="right", rotation_mode="anchor")
    plt.yticks(fontsize=12)
    plt.grid(True, color="grey", linestyle="--", linewidth=0.5)
    plt.tight_layout()
    saveFigureToPdf(fig, pdf)

    # 4. Sales distributions
    for fig in prepareSalesDistributions(df, days=days):
        saveFigureToPdf(fig, pdf)

    # Close the PDF
    pdf.close()


In [ ]:

dfSalesMerged = createMergedDataFrameFromExcelMatch(directory= DIRECTORY, keyword= KEYWORDS, columns= SALES_COLUMNS, saveToExcel= False, outputPath= OUTPUT_PATH, outputFileName= OUTPUT_FILE_NAME)

dfSalesMerged = cleanDateColumn(dfSalesMerged, DATE)

generateSalesEDA(dfSalesMerged, "../output/salesEDAReport.pdf", days=90)

#saveDataFrameToExcel(dfSalesMerged, OUTPUT_PATH, OUTPUT_FILE_NAME)


Could not determine dtype for column 1, falling back to string
Could not determine dtype for column 2, falling back to string
Could not determine dtype for column 3, falling back to string
Could not determine dtype for column 10, falling back to string
Could not determine dtype for column 12, falling back to string
Could not determine dtype for column 13, falling back to string
Could not determine dtype for column 20, falling back to string
Could not determine dtype for column 21, falling back to string
Could not determine dtype for column 28, falling back to string
Could not determine dtype for column 29, falling back to string
Could not determine dtype for column 30, falling back to string
Could not determine dtype for column 31, falling back to string
Could not determine dtype for column 32, falling back to string
Could not determine dtype for column 33, falling back to string
Could not determine dtype for column 37, falling back to string
Could not determine dtype for column 38, fa